In [1]:
import sys  
sys.path.insert(0, '/Users/arturo/Downloads/')

from Funciones import *

In [2]:
import pyspark
from pyspark.sql import SparkSession
from Funciones import *
# from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
# from pyspark.sql.types import ArrayType, DoubleType, BooleanType
# from pyspark.sql.functions import col,array_contains



In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/20 23:24:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Datos

In [4]:
fi4 = spark.read.csv("DATA_SIC_ENTIDADES_PUBLICAS_AYUDAS_ESP.csv", sep=";", header=True)
a12 = spark.read.csv("DATA_RELACION_ENTIDADES_MAESTRO.csv", sep=";", header=True)
fi5 = spark.read.csv("DATA_SIC_AYUDA_ESP_UO_PB_J_MAESTRO.csv", sep=";", header=True)

# Cruce 1

In [5]:
cruce1 = fi5.join(a12,fi5.Id ==  a12.Id_NIVEL_0,"inner")

In [6]:
cruce1 = cruce1.withColumn("Similitud", 
                           udf_Distance_ratcliff_obershelp(
                               UDF_normalizarTexto(cruce1.NOMBRE_ENTIDAD_NIVEL_1),
                               cruce1.Centro_Norm
                           )) 

# DATA_SIC_ENTIDADES_CENTROS_PB

In [7]:
maxId = fi4.withColumn("Id", fi4.Id.cast('int')).select("Id").rdd.max()[0]

In [8]:
data = cruce1.filter("Similitud <0.875")\
             .withColumn("CIF",lit(None).cast(StringType()))\
             .withColumn("Entidad_Norm",col("Centro_Norm"))\
             .withColumn("Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("ID_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("NIF_COD",lit(None).cast(StringType()))\
             .withColumn("ACRONIMO",lit(None).cast(StringType()))\
             .withColumn("NOMBRE_ENTIDAD",lit(None).cast(StringType()))\
             .withColumn("Nombre_Entidad_Mostrar",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N1_1",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_N2_1",lit(None).cast(StringType()))\
             .withColumn("DIRECCION_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_POSTAL",lit(None).cast(StringType()))\
             .withColumn("COD_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("COD_CCAA",lit(None).cast(StringType()))\
             .withColumn("CCAA",lit(None).cast(StringType()))\
             .withColumn("ENLACE_WEB",lit(None).cast(StringType()))\
             .withColumn("SOMMA",lit(None).cast(StringType()))\
             .withColumn("TIPO_ENTIDAD_REGIONAL",lit(None).cast(StringType()))\
             .withColumn("ESTADO_x",lit(None).cast(StringType()))\
             .withColumn("CodigoInvente",lit(None).cast(StringType()))\
             .withColumn("DenominacionSocial",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Codigo",lit(None).cast(StringType()))\
             .withColumn("FormaJuridica_Descripcion",lit(None).cast(StringType()))\
             .withColumn("NIF",lit(None).cast(StringType()))\
             .withColumn("codigoDir3",lit(None).cast(StringType()))\
             .withColumn("codigoOrigen",lit(None).cast(StringType()))\
             .withColumn("Provincia_Codigo",lit(None).cast(StringType()))\
             .withColumn("C_ID_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA",lit(None).cast(StringType()))\
             .withColumn("C_ID_NIVEL_ADMON",lit(None).cast(StringType()))\
             .withColumn("C_ID_TIPO_ENT_PUBLICA",lit(None).cast(StringType()))\
             .withColumn("N_NIVEL_JERARQUICO",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_SUPERIOR",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_UD_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("B_SW_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_DEP_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_DNM_UD_ORGANICA_EDP_PRINCIPAL",lit(None).cast(StringType()))\
             .withColumn("C_ID_ESTADO",lit(None).cast(StringType()))\
             .withColumn("D_VIG_ALTA_OFICIAL",lit(None).cast(StringType()))\
             .withColumn("NIF_CIF",lit(None).cast(StringType()))\
             .withColumn("C_ID_AMB_PROVINCIA",lit(None).cast(StringType()))\
             .withColumn("C_DESC_PROV",lit(None).cast(StringType()))\
             .withColumn("CONTACTOS",lit(None).cast(StringType()))\
             .withColumn("List_Entidad_Norm",col("Centro_Norm"))\
             .withColumn("List_Provincia_Entidad",col("Provincia_Centro"))\
             .withColumn("List_CIF",lit(None).cast(StringType()))

In [9]:
data = data.dropDuplicates(["Entidad_Norm", "Provincia_Entidad"])
data = data.drop('Id')

In [10]:
Index = list(range(maxId+ 1,maxId + data.count() + 1))
Index = spark.createDataFrame(Index, IntegerType())
Index = Index.withColumnRenamed("value", "Id")

In [11]:
data = data.withColumn("Dummy", monotonically_increasing_id())
Index = Index.withColumn("Dummy", monotonically_increasing_id())
data = data.join(Index, "Dummy", "outer").drop("Dummy")

In [12]:
data_1 = data.select("Id",
                    "CIF",
                    "Entidad_Norm",
                    "Provincia_Entidad",
                    "ID_ENTIDAD",
                    "NIF_COD",
                    "ACRONIMO",
                    "NOMBRE_ENTIDAD",
                    "Nombre_Entidad_Mostrar",
                    "TIPO_ENTIDAD_N1_1",
                    "TIPO_ENTIDAD_N2_1",
                    "DIRECCION_POSTAL",
                    "COD_POSTAL",
                    "COD_PROVINCIA",
                    "PROVINCIA",
                    "COD_CCAA",
                    "CCAA",
                    "ENLACE_WEB",
                    "SOMMA",
                    "TIPO_ENTIDAD_REGIONAL",
                    "ESTADO_x",
                    "CodigoInvente",
                    "DenominacionSocial",
                    "FormaJuridica_Codigo",
                    "FormaJuridica_Descripcion",
                    "NIF",
                    "codigoDir3",
                    "codigoOrigen",
                    "Provincia_Codigo",
                    "C_ID_UD_ORGANICA",
                    "C_DNM_UD_ORGANICA",
                    "C_ID_NIVEL_ADMON",
                    "C_ID_TIPO_ENT_PUBLICA",
                    "N_NIVEL_JERARQUICO",
                    "C_ID_DEP_UD_SUPERIOR",
                    "C_DNM_UD_ORGANICA_SUPERIOR",
                    "C_ID_DEP_UD_PRINCIPAL",
                    "C_DNM_UD_ORGANICA_PRINCIPAL",
                    "B_SW_DEP_EDP_PRINCIPAL",
                    "C_ID_DEP_EDP_PRINCIPAL",
                    "C_DNM_UD_ORGANICA_EDP_PRINCIPAL",
                    "C_ID_ESTADO",
                    "D_VIG_ALTA_OFICIAL",
                    "NIF_CIF",
                    "C_ID_AMB_PROVINCIA",
                    "C_DESC_PROV",
                    "CONTACTOS",
                    "List_Entidad_Norm",
                    "List_Provincia_Entidad",
                    "List_CIF")

In [23]:
data_1.write.option("header",True).format("csv").save("/Nombre_Intermedio_cruce1_entidades/")

In [ ]:
data_1.write.format("parquet").save("/Nombre_Intermedio_cruce1_entidades_parquet/")

# Nombre_Output_investigadores

In [13]:
data_2 = cruce1.filter("Id_NIVEL_0 is not NULL and Id is not NULL and Similitud >=0.875")\
        .withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id_NIVEL_1"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(fi5.columns + ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [14]:
data_3 = data.withColumn("Final_Score", col("Similitud"))\
        .withColumn("Id_Centro", col("Id"))\
        .withColumn("Centro_Nombre_match", col("NOMBRE_ENTIDAD_NIVEL_1"))\
        .withColumn("Centro_Provincia_match", col("COD_PROVINCIA_NIVEL_1"))\
        .withColumn("Similitud_Centro", col("Similitud"))\
        .select(fi5.columns + ["Final_Score","Id_Centro", "Centro_Nombre_match", "Centro_Provincia_match", "Similitud_Centro"])


In [18]:
result = data_2.union(data_3)

In [20]:
result.write.option("header",True).format("csv").save("/Users/arturo/repos/repos/test1/data-eng-scripts/test")

22/07/20 23:34:34 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/07/20 23:34:35 ERROR FileOutputCommitter: Mkdirs failed to create file:/Nombre_Output_investigadores/_temporary/0


22/07/20 23:40:30 ERROR Executor: Exception in task 10.0 in stage 20.0 (TID 46)
java.io.IOException: Mkdirs failed to create file:/Nombre_Output_investigadores/_temporary/0/_temporary/attempt_202207202340292600958027802038332_0020_m_000010_46 (exists=false, cwd=file:/Users/arturo/Documents/AVALON)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at org.apache.spark.sql.execution.datasources.csv.CsvOutputWriter.<init>(CsvOutputWriter.scala:38)
	at org.apache.spark.sql.execution

xecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:750)
22/07/20 23:40:30 ERROR Executor: Exception in task 11.0 in stage 20.0 (TID 47)
java.io.IOException: Mkdirs failed to create file:/Nombre_Output_investigadores/_temporary/0/_temporary/attempt_20220720234029384137653949799326_0020_m_000011_47 (exists=false, cwd=file:/Users/arturo/Documents/AVALON)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at org.apache.spark.sql.execution.datasource

Py4JJavaError: An error occurred while calling o674.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:638)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:278)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:98)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$6(SQLExecution.scala:109)
	at org.ap

In [ ]:
result.write.format("parquet").save("/Nombre_Output_investigadores/")

22/07/20 23:40:32 WARN TaskSetManager: Lost task 0.0 in stage 20.0 (TID 48) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)


22/07/20 23:40:32 WARN TaskSetManager: Lost task 7.0 in stage 20.0 (TID 55) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 1.0 in stage 20.0 (TID 49) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)


22/07/20 23:40:33 WARN TaskSetManager: Lost task 9.0 in stage 20.0 (TID 57) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 8.0 in stage 20.0 (TID 56) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 2.0 in stage 20.0 (TID 50) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 3.0 in stage 20.0 (TID 51) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 5.0 in stage 20.0 (TID 53) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 6.0 in stage 20.0 (TID 54) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)
22/07/20 23:40:33 WARN TaskSetManager: Lost task 4.0 in stage 20.0 (TID 52) (10.0.0.103 executor driver): TaskKilled (Stage cancelled)


In [ ]:
data = data.withColumn("Id", sequence(lit(maxId+1), lit(maxId+data.count())))

In [18]:
a12.withColumn("range", array((1 to 100).map(lit(_)): _*)).show()

SyntaxError: invalid syntax (2611963741.py, line 1)

In [ ]:
data.write.csv("/Users/arturo/repos/repos/test1/data-eng-scripts/test")

In [16]:
data.select("Id").show()

+---+
| Id|
+---+
|833|
|237|
|856|
|930|
|871|
|237|
|849|
|853|
|237|
|237|
|237|
|841|
|847|
|847|
|886|
|861|
|833|
|244|
|859|
|247|
+---+
only showing top 20 rows



In [39]:
# list  of employee data
data = [["1", "sravan", "company 1"],
        ["2", "ojaswi", "company 1"],
        ["3", "rohith", "company 2"],
        ["4", "sridevi", "company 1"],
        ["5", "bobby", "company 1"]]
 
# specify column names
columns = ['ID', 'NAME', 'Company']
 
# creating a dataframe from the lists of data
dataframe = spark.createDataFrame(data, columns)

In [61]:
maxid = 1000

In [70]:

mylist = list(range(maxid+ 1,maxid + dataframe.count() + 1))

# notice the parens after the type name
index = spark.createDataFrame(mylist, IntegerType())
index = index.withColumnRenamed("value", "Id")

index.show()

+----+
|  Id|
+----+
|1001|
|1002|
|1003|
|1004|
|1005|
+----+



In [64]:
dataframe = dataframe.withColumn("Dummy", monotonically_increasing_id())
index = index.withColumn("Dummy", monotonically_increasing_id())
df3 = dataframe.join(index, "Dummy", "outer").drop("Dummy")

In [65]:
df3.show()

+---+-------+---------+-----+
| ID|   NAME|  Company|value|
+---+-------+---------+-----+
|  1| sravan|company 1| 1001|
|  2| ojaswi|company 1| 1002|
|  3| rohith|company 2| 1003|
|  4|sridevi|company 1| 1004|
|  5|  bobby|company 1| 1005|
+---+-------+---------+-----+



In [55]:
dataframe.withColumn("Index", col(index.select("value")))

AttributeError: 'DataFrame' object has no attribute '_get_object_id'

In [52]:
index.show()

+-----+
|value|
+-----+
|   11|
|   12|
|   13|
|   14|
|   15|
+-----+



In [ ]:
result = df_1_new.union(df_2_new)

In [32]:
dataframe.show()

+---+-------+---------+--------------------+
| ID|   NAME|  Company|               YEARS|
+---+-------+---------+--------------------+
|  1| sravan|company 1|[11, 12, 13, 14, 15]|
|  2| ojaswi|company 1|[11, 12, 13, 14, 15]|
|  3| rohith|company 2|[11, 12, 13, 14, 15]|
|  4|sridevi|company 1|[11, 12, 13, 14, 15]|
|  5|  bobby|company 1|[11, 12, 13, 14, 15]|
+---+-------+---------+--------------------+



In [26]:
dataframe.withColumn("Idl", udf(lambda id: dates[id])).show()

TypeError: col should be Column

In [ ]:
df = df.repartition(1).withColumn("YEARS",  udf(lambda id: dates[id])

In [24]:
sequence(lit(maxId+1), lit(maxId+dataframe.count()))

Column<'sequence(8199, 8203)'>

In [65]:
fi4.count()

8210

8198

In [55]:
fi4.select("id").show()

+----+
|  id|
+----+
|2668|
|4782|
|4589|
|3515|
|2891|
|7281|
|2635|
| 547|
| 352|
|8193|
|5328|
|6552|
|1491|
|1375|
|6408|
|2344|
|2119|
|8102|
|1528|
|6590|
+----+
only showing top 20 rows



In [48]:
list(set(final.columns) - set(fi4.columns))

[]

In [49]:
len(final.columns)

49

In [50]:
len(fi4.columns)

50

In [45]:
pd.DataFrame(final.columns)[0].value_counts()

C_ID_DEP_UD_PRINCIPAL              2
CIF                                1
codigoDir3                         1
Provincia_Codigo                   1
C_ID_UD_ORGANICA                   1
C_DNM_UD_ORGANICA                  1
C_ID_NIVEL_ADMON                   1
C_ID_TIPO_ENT_PUBLICA              1
N_NIVEL_JERARQUICO                 1
C_ID_DEP_UD_SUPERIOR               1
C_DNM_UD_ORGANICA_SUPERIOR         1
C_DNM_UD_ORGANICA_PRINCIPAL        1
B_SW_DEP_EDP_PRINCIPAL             1
C_ID_DEP_EDP_PRINCIPAL             1
C_DNM_UD_ORGANICA_EDP_PRINCIPAL    1
C_ID_ESTADO                        1
D_VIG_ALTA_OFICIAL                 1
NIF_CIF                            1
C_ID_AMB_PROVINCIA                 1
C_DESC_PROV                        1
CONTACTOS                          1
List_Entidad_Norm                  1
List_Provincia_Entidad             1
codigoOrigen                       1
NIF                                1
Entidad_Norm                       1
FormaJuridica_Descripcion          1
P

In [16]:
fi4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [102]:
final.printSchema()

root
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = true)
 |-- FormaJuridica_Codigo: string (nullable = true)
 |-- FormaJu

In [51]:
fi4.printSchema()

root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [6]:
cruce1.columns

['Id',
 'Entidad',
 'Origen_Solicitud',
 'Identificadores_Origen',
 'Entidad_Norm',
 'CIF',
 'CIF_validacion',
 'Centro',
 'Centro_Norm',
 'Tipo',
 'Provincia_Entidad',
 'Pais_Entidad',
 'Provincia_Centro',
 'PIC',
 'Tipo_Persona',
 'Entidad_Match',
 'Provincia_Match',
 'Match',
 'Id_NIVEL_1',
 'Id_NIVEL_0',
 'ID_ENTIDAD_NIVEL_0',
 'ID_ENTIDAD_NIVEL_1',
 'Jerarquia',
 'NIF_COD_NIVEL_0',
 'ACRONIMO_NIVEL_0',
 'NOMBRE_ENTIDAD_NIVEL_0',
 'Nombre_Entidad_Mostrar_NIVEL_0',
 'TIPO_ENTIDAD_N1_1_NIVEL_0',
 'TIPO_ENTIDAD_N2_1_NIVEL_0',
 'DIRECCION_POSTAL_NIVEL_0',
 'COD_POSTAL_NIVEL_0',
 'COD_PROVINCIA_NIVEL_0',
 'PROVINCIA_NIVEL_0',
 'COD_CCAA_NIVEL_0',
 'CCAA_NIVEL_0',
 'ENLACE_WEB_NIVEL_0',
 'SOMMA_NIVEL_0',
 'TIPO_ENTIDAD_REGIONAL_NIVEL_0',
 'ESTADO_x_NIVEL_0',
 'NIF_COD_NIVEL_1',
 'ACRONIMO_NIVEL_1',
 'NOMBRE_ENTIDAD_NIVEL_1',
 'Nombre_Entidad_Mostrar_NIVEL_1',
 'TIPO_ENTIDAD_N1_1_NIVEL_1',
 'TIPO_ENTIDAD_N2_1_NIVEL_1',
 'DIRECCION_POSTAL_NIVEL_1',
 'COD_POSTAL_NIVEL_1',
 'COD_PROVINCIA_NI

In [88]:
setA = set(cruce1.columns)
setB = set(fi4.columns)

print(setA & setB)

{'CIF', 'Entidad_Norm', 'Id', 'Provincia_Entidad'}


['Id',
 'CIF',
 'Entidad_Norm',
 'Provincia_Entidad',
 'ID_ENTIDAD',
 'NIF_COD',
 'ACRONIMO',
 'NOMBRE_ENTIDAD',
 'Nombre_Entidad_Mostrar',
 'TIPO_ENTIDAD_N1_1',
 'TIPO_ENTIDAD_N2_1',
 'DIRECCION_POSTAL',
 'COD_POSTAL',
 'COD_PROVINCIA',
 'PROVINCIA',
 'COD_CCAA',
 'CCAA',
 'ENLACE_WEB',
 'SOMMA',
 'TIPO_ENTIDAD_REGIONAL',
 'ESTADO_x',
 'CodigoInvente',
 'DenominacionSocial',
 'FormaJuridica_Codigo',
 'FormaJuridica_Descripcion',
 'NIF',
 'codigoDir3',
 'codigoOrigen',
 'Provincia_Codigo',
 'C_ID_UD_ORGANICA',
 'C_DNM_UD_ORGANICA',
 'C_ID_NIVEL_ADMON',
 'C_ID_TIPO_ENT_PUBLICA',
 'N_NIVEL_JERARQUICO',
 'C_ID_DEP_UD_SUPERIOR',
 'C_DNM_UD_ORGANICA_SUPERIOR',
 'C_ID_DEP_UD_PRINCIPAL',
 'C_DNM_UD_ORGANICA_PRINCIPAL',
 'B_SW_DEP_EDP_PRINCIPAL',
 'C_ID_DEP_EDP_PRINCIPAL',
 'C_DNM_UD_ORGANICA_EDP_PRINCIPAL',
 'C_ID_ESTADO',
 'D_VIG_ALTA_OFICIAL',
 'NIF_CIF',
 'C_ID_AMB_PROVINCIA',
 'C_DESC_PROV',
 'CONTACTOS',
 'List_Entidad_Norm',
 'List_Provincia_Entidad',
 'List_CIF']

In [14]:
cruce1.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Entidad: string (nullable = true)
 |-- Origen_Solicitud: string (nullable = true)
 |-- Identificadores_Origen: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- CIF_validacion: string (nullable = true)
 |-- Centro: string (nullable = true)
 |-- Centro_Norm: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- Pais_Entidad: string (nullable = true)
 |-- Provincia_Centro: string (nullable = true)
 |-- PIC: string (nullable = true)
 |-- Tipo_Persona: string (nullable = true)
 |-- Entidad_Match: string (nullable = true)
 |-- Provincia_Match: string (nullable = true)
 |-- Match: string (nullable = true)
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |--

In [12]:
fi4.printSchema()


root
 |-- Id: string (nullable = true)
 |-- CIF: string (nullable = true)
 |-- Entidad_Norm: string (nullable = true)
 |-- Provincia_Entidad: string (nullable = true)
 |-- ID_ENTIDAD: string (nullable = true)
 |-- NIF_COD: string (nullable = true)
 |-- ACRONIMO: string (nullable = true)
 |-- NOMBRE_ENTIDAD: string (nullable = true)
 |-- Nombre_Entidad_Mostrar: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1: string (nullable = true)
 |-- DIRECCION_POSTAL: string (nullable = true)
 |-- COD_POSTAL: string (nullable = true)
 |-- COD_PROVINCIA: string (nullable = true)
 |-- PROVINCIA: string (nullable = true)
 |-- COD_CCAA: string (nullable = true)
 |-- CCAA: string (nullable = true)
 |-- ENLACE_WEB: string (nullable = true)
 |-- SOMMA: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL: string (nullable = true)
 |-- ESTADO_x: string (nullable = true)
 |-- CodigoInvente: string (nullable = true)
 |-- DenominacionSocial: string (nullable = t

In [21]:
a12.printSchema()

root
 |-- Id_NIVEL_1: string (nullable = true)
 |-- Id_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- ID_ENTIDAD_NIVEL_1: string (nullable = true)
 |-- Jerarquia: string (nullable = true)
 |-- NIF_COD_NIVEL_0: string (nullable = true)
 |-- ACRONIMO_NIVEL_0: string (nullable = true)
 |-- NOMBRE_ENTIDAD_NIVEL_0: string (nullable = true)
 |-- Nombre_Entidad_Mostrar_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N1_1_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_N2_1_NIVEL_0: string (nullable = true)
 |-- DIRECCION_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_POSTAL_NIVEL_0: string (nullable = true)
 |-- COD_PROVINCIA_NIVEL_0: string (nullable = true)
 |-- PROVINCIA_NIVEL_0: string (nullable = true)
 |-- COD_CCAA_NIVEL_0: string (nullable = true)
 |-- CCAA_NIVEL_0: string (nullable = true)
 |-- ENLACE_WEB_NIVEL_0: string (nullable = true)
 |-- SOMMA_NIVEL_0: string (nullable = true)
 |-- TIPO_ENTIDAD_REGIONAL_NIVEL_0: string (nullab